## Загрузка датасета

In [46]:
from IPython.display import clear_output
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

/home/workspace/llama_tests/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
ds = load_dataset("kuznetsoffandrey/sberquad")

## Загрузка модели

In [93]:
model_name = 'DiTy/bi-encoder-russian-msmarco'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = SentenceTransformer(model_name).to('cuda')

## Удаление дубликатов из контекста и создание эмбеддингов контекста

In [49]:
context_sentences = list(set(ds["train"]['context']))

In [5]:
context_embeddings = model.encode(context_sentences)

## Сохранение вычисленных эмбеддингов

In [50]:
save_path = "data/context_embeddings.npy"


In [7]:
try:
    np.load(save_path)
    print('Loaded!')
except Exception:
    np.save(save_path, context_embeddings)
    print("Saved!")


Loaded!


In [51]:
context_embeddings = np.load(save_path)

## Создание экземляра базы ChromaDB для хранения эмббедингов

In [52]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb
from chromadb.utils import embedding_functions

In [53]:
client = chromadb.Client()
collection = client.get_or_create_collection(name='sberquad_rag')

In [54]:
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
collection.add(
    documents=context_sentences,
    embeddings=context_embeddings.tolist(),
    ids=[str(i) for i in range(len(context_sentences))]
)

## Результаты

In [56]:
questions = ds["validation"]['question']
correct_contexts = ds["validation"]['context']

In [64]:
from tqdm import tqdm
import pprint

In [92]:
questions = ds["train"]['question'][0]
correct_contexts = ds["train"]['context'][0]

result = collection.query(
        query_embeddings=model.encode(questions).tolist(),
        n_results=10,
    )
result['documents']

[['Собственный капитал банка вырос более чем на 22 %: на конец года он достиг 9,2 млрд рублей. Общая величина активов Банка на 1 января 2014 года выросла на 12,5 % по отношению к аналогичному показателю на 1 января 2013 г. и превысила 81 млрд рублей. По итогам года кредитный портфель банка вырос на 11,75 % и превысил 50 млрд рублей. Размер кредитов, выданных физическим лицам, вырос на 28 %, а вклады населения превысили 16,6 млрд рублей, увеличившись по сравнению с прошлым годом на почти на 17 %. За год Банк открыл 7 новых дополнительных отделений. Их общее число достигло 62.',
  'Истинным родоначальником эпохи Возрождения в литературе принято считать итальянского поэта Данте Алигьери (1265—1321), который истинно раскрыл сущность людей того времени в своем произведении под названием Комедия , которое впоследствии будет названо Божественной комедией . Этим названием потомки явили своё восхищение грандиозным созданием Данте. В литературе Возрождения наиболее полно выразились гуманистическ

In [19]:
print(correct_retrievals)


2


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [44]:

import llama_cpp
model_name = 'models/meta-llama-3.1-8b-instruct.Q6_K.gguf'
class RAG:
    def __init__(self, model_name: str) -> None:
        self.rag_model = llama_cpp.Llama(
            model_path=model_name,
            split_mode=llama_cpp.LLAMA_SPLIT_MODE_LAYER,
            n_gpu_layers=-1,
            offload_kqv=True,
        )

    def get_response(self, context: str, question: str) -> str:
        response = self.rag_model.create_chat_completion(messages = [
            {
                "role": "system", "content":
        """
        Используя информацию, содержащуюся в контексте, дай полный ответ на вопрос. Отвечай только на поставленный вопрос, ответ должен соответствовать вопросу. Если ответ не может быть выведен из контекста, не отвечай.
        Оформляй красиво итоговый ответ
        """,
            },
            {
                "role": "user", "content":
        f"""
        Контекст: {context}
        ---
        Вот вопрос на который тебе надо ответить.
        Вопрос: {question}
        """,
            },
        ], temperature=0.2, top_p=0.80)
        return response['choices'][0]['message']['content']

In [45]:
RAG(model_name).get_response('В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.','чем представлены органические остатки?', )

llama_model_loader: loaded meta data with 29 key-value pairs and 292 tensors from models/meta-llama-3.1-8b-instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = llama3.1
llama_model_loader: - kv   7: 

'В протерозойских отложениях органические остатки представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных, скоплениями графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum, нитевидными водорослями, грибными нитями и формами, близкими современным кокколитофоридам, а также железистыми продуктами жизнедеятельности бактерий.'